In [6]:
import pandas as pd
import numpy as np
data=pd.read_csv('data.csv')
data

,1102,710,-655,0
0,145,-105,882,0
1,62,305,741,0
2,78,50,792,0
3,-58,19,1031,0
4,-109,-35,1082,0
...,...,...,...,...
132698,-305,474,659,9
132699,-309,478,670,9
132700,-325,494,674,9
132701,-325,509,678,9


In [7]:
df=pd.DataFrame(data)
df.columns=['x-axis','y-axis','z-axis','number']
df

,x-axis,y-axis,z-axis,number
0,145,-105,882,0
1,62,305,741,0
2,78,50,792,0
3,-58,19,1031,0
4,-109,-35,1082,0
...,...,...,...,...
132698,-305,474,659,9
132699,-309,478,670,9
132700,-325,494,674,9
132701,-325,509,678,9


In [8]:
df['number'].value_counts()

44    13315
3     13300
8     13296
2     13280
5     13279
0     13270
6     13257
7     13243
1     13233
9     13230
Name: number, dtype: int64

In [9]:
df['number'] = df['number'].replace(44,4)
df['number'].value_counts()


4    13315
3    13300
8    13296
2    13280
5    13279
0    13270
6    13257
7    13243
1    13233
9    13230
Name: number, dtype: int64

## Standardized data

In [10]:
from sklearn.preprocessing import StandardScaler
x=df[['x-axis','y-axis','z-axis']]
y=df['number']
scaler=StandardScaler()
X=scaler.fit_transform(x)


In [11]:
scaled_X=pd.DataFrame(data=X,columns=['x-axis','y-axis','z-axis'])
scaled_X['number']=y.values

scaled_X

,x-axis,y-axis,z-axis,number
0,0.615454,-0.445042,0.223163,0
1,0.318892,0.355091,-0.123441,0
2,0.376060,-0.142553,0.001926,0
3,-0.109873,-0.203051,0.589434,0
4,-0.292097,-0.308434,0.714801,0
...,...,...,...,...
132698,-0.992413,0.684902,-0.325013,9
132699,-1.006705,0.692708,-0.297973,9
132700,-1.063873,0.723933,-0.288140,9
132701,-1.063873,0.753206,-0.278307,9


In [7]:
df

,x-axis,y-axis,z-axis,number
0,145,-105,882,0
1,62,305,741,0
2,78,50,792,0
3,-58,19,1031,0
4,-109,-35,1082,0
...,...,...,...,...
132698,-305,474,659,9
132699,-309,478,670,9
132700,-325,494,674,9
132701,-325,509,678,9


In [12]:
import scipy.stats as stats

In [13]:
fs=20
frame_size=fs*9
hop_size=fs*2

In [14]:
def get_frames(df,frame_size,hop_size):
    N_FEATURES=3
    
    frames=[]
    lables=[]
    for i in range(0,len(df)-frame_size,hop_size):
        x=df['x-axis'].values[i: i+frame_size]
        y=df['y-axis'].values[i: i+frame_size]
        z=df['z-axis'].values[i: i+frame_size]   
        
        lable = stats.mode(df['number'][i: i+frame_size])[0][0]
        frames.append([x,y,z])
        lables.append(lable)
    frames=np.asarray(frames).reshape(-1,frame_size,N_FEATURES)
    lables=np.asarray(lables)
    
    return frames ,lables

In [15]:
X_x,Y_y=get_frames(df,frame_size,hop_size)

/var/folders/tn/c2cbg8n95ws2j0pgvtymb0cc0000gn/T/ipykernel_92993/25944643.py:11: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  lable = stats.mode(df['number'][i: i+frame_size])[0][0]


In [16]:
X_x.shape , Y_y.shape

((3314, 180, 3), (3314,))

In [17]:
from sklearn.model_selection import KFold, cross_val_score, train_test_split


In [18]:
X_train , X_test, y_train, y_test = train_test_split(X_x,Y_y,test_size=0.2,random_state=0,stratify=Y_y)

In [19]:
X_test.shape,X_train.shape,y_test.shape,y_train.shape

((663, 180, 3), (2651, 180, 3), (663,), (2651,))

In [20]:
X_test= X_test.reshape(663, 180, 3, 1)
X_train=X_train.reshape(2651, 180, 3, 1)

In [21]:
X_test.shape,X_train.shape

((663, 180, 3, 1), (2651, 180, 3, 1))

### 2D CNN Model

In [32]:
from keras.models import Sequential
from tensorflow.keras.layers import Conv2D ,Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam

In [30]:
model=Sequential()
model.add(Conv2D(16, (2,2), activation = 'relu', input_shape= X_train[0].shape))
model.add(Dropout(0.1))

model.add(Conv2D(32, (2,2), activation = 'relu'))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(10, activation='softmax'))

In [41]:
model.compile(optimizer=Adam(learning_rate = 0.001), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [45]:

history = model.fit(X_train, y_train, epochs = 2, batch_size=5, validation_data= (X_test, y_test),verbose=1)

Epoch 1/2


2023-02-06 22:47:39.373750: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:418 : NOT_FOUND: could not find registered platform with id: 0x17569e2a0
2023-02-06 22:47:39.373789: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:418 : NOT_FOUND: could not find registered platform with id: 0x17569e2a0
2023-02-06 22:47:39.457746: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:418 : NOT_FOUND: could not find registered platform with id: 0x17569e2a0
2023-02-06 22:47:39.457782: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:418 : NOT_FOUND: could not find registered platform with id: 0x17569e2a0
2023-02-06 22:47:39.625684: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:418 : NOT_FOUND: could not find registered platform with id: 0x17569e2a0
2023-02-06 22:47:39.625723: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed a

NotFoundError: Graph execution error:

Detected at node 'StatefulPartitionedCall_6' defined at (most recent call last):
    File "/Users/rahuls/anaconda3/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Users/rahuls/anaconda3/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/Users/rahuls/anaconda3/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/rahuls/anaconda3/lib/python3.9/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/Users/rahuls/anaconda3/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/Users/rahuls/anaconda3/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/Users/rahuls/anaconda3/lib/python3.9/asyncio/base_eve